In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [2]:
path = '../data/'
savepath = '../data/'
images = glob.glob(path+'new_data/*.npy')

In [ ]:
tile_sizes = np.loadtxt(path+'tile_sizes.txt', dtype='int')
images_sampled = {}
for tile in tile_sizes:
    if tile[2] > 200000:
        for i in range(30):
            images_sampled.setdefault(tile[0]*30+i, []).append(tile[1])

In [ ]:
def myGenerator(batch_size):
    while True:
        index_list = random.sample(images_sampled.keys(), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = path+'sources/new_data/frame'+str(i)+'.npy'
            frame = np.load(frame)
            tile_index = random.choice(images_sampled[i])
            
            temp  = imresize(frame[tile_index], (48, 48))
            temp  = imresize(temp, (384, 384))
            
            alldata_x.append(temp)
            alldata_y.append(frame[tile_index])
        
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        alldata_x = alldata_x.astype(np.float32)/255.0
        
        yield alldata_x, alldata_y

#x = myGenerator(10)
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [ ]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 384
        self.img_cols = 384
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(384, 384, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        input_size = (384,384,3)
        inputs = Input(input_size)


        conv1 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = LeakyReLU()(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)



        conv2 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv2 = LeakyReLU()(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        conv3 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv3 = LeakyReLU()(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)

        concat1 = add([conv1, conv3])



        conv4 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(concat1)
        conv4 = LeakyReLU()(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)

        conv5 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
        conv5 = LeakyReLU()(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)

        concat2 = add([conv5, concat1])



        conv6 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(concat2)
        conv6 = LeakyReLU()(conv6)
        conv6 = BatchNormalization(momentum=0.8)(conv6)

        conv7 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
        conv7 = LeakyReLU()(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        concat3 = add([conv7, concat2])



        conv8 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(concat3)
        conv8 = LeakyReLU()(conv8)
        conv8 = BatchNormalization(momentum=0.8)(conv8)

        conv9 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv9 = LeakyReLU()(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)
        

        conv10 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = LeakyReLU()(conv10)
        conv10 = BatchNormalization(momentum=0.8)(conv10)

        concat4 = add([conv10, conv1])
        
        conv11 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(concat4)
        conv11 = LeakyReLU()(conv11)
        conv11 = BatchNormalization(momentum=0.8)(conv11)
        
        conv12 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv11)
        conv12 = PReLU()(conv12)
        conv12 = BatchNormalization(momentum=0.8)(conv12)


        out = Conv2D(3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv12)
        out = PReLU()(out)
        out = BatchNormalization(momentum=0.8)(out)


        model = Model(input = inputs, output = out)
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, y_train)
            print ("Epoch ", epoch, " G loss ", g_loss)
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, y_train = next(myGenerator(10))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        y_train = (0.5 * y_train + 0.5)*255
        y_train = y_train.astype(int)
        X_train = X_train*255
        X_train = X_train.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6], gen_imgs[7], gen_imgs[8], gen_imgs[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], y_train[5], y_train[6], y_train[7], y_train[8], y_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)
        
        combined = np.array([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5], X_train[6], X_train[7], X_train[8], X_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_lowres.png", combined)

In [ ]:
cgan = CGAN()
cgan.build_autoencoder()
cgan.train_generator_autoencoder(100000, 6, 100)

In [ ]:
for i in range(1000):
    x = myGenerator(1)
    xtest, ytest = next(x)
    pred = cgan.generator.predict(xtest)
    pred = pred*127.5 + 127.5
    pred = pred.astype(int)
    #plt.imshow(pred[0])
    #plt.show()
    ytest = ytest*127.5+127.5
    ytest = ytest.astype(int)
    #plt.imshow(ytest[0])
    #plt.show()
    imsave(path+'test/frame_pred'+str(i)+'.png', pred[0])
    imsave(path+'test/frame_real'+str(i)+'.png', ytest[0])

In [3]:
import numpy 
import math
import cv2

def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [6]:
data = []
for i in range(1000):
    img1 = cv2.imread(path+'test/frame_real'+str(i)+'.png')
    img2 = cv2.imread(path+'test/frame_pred'+str(i)+'.png')
    d = psnr(img1, img2)
    data.append(d)
    print (d)
print('PSNR:', np.mean(data))

27.9395259960662
27.445484068009264
27.871500982818883
29.340316338325923
28.042847687154136
27.913226644603863
28.04647503714981
28.246360358293565
27.983441016266415
27.79587888365487
28.040132911204616
27.716075009664117
27.99362390358164
27.486075610202178
27.531491832415487
27.8294002022026
27.38297666274205
27.656603069050316
27.87625036097623
27.344529781109067
27.583646693633437
28.91553263708999
27.6893404899025
28.053988453925296
28.222040288998848
27.29470625309311
28.073733078377106
27.497426031584375
28.19086944040455
27.99533425507245
28.16344068729603
28.141625354877963
27.881701447706146
27.285481144004006
28.04791447777395
28.084168704519577
28.172907120131118
27.519662620035184
27.682375105292
28.064950355325532
27.88576546960286
27.359836838126537
26.911637413644762
28.195732707945474
27.882695329809156
27.899410916724115
28.05475918773012
27.756196021282328
28.88704406632602
28.359027238820097
27.096180285721523
28.07228060068867
28.775464177929514
28.49549892109571

30.045420132274508
28.95166751739735
27.649262085137295
29.985890916545795
27.134315645661594
28.375601104261676
27.682899935623183
28.486768477531438
28.792383510161468
29.820062601439517
28.114592235321062
30.301089478982597
27.69075605793724
28.991953255200865
27.797526547193165
30.086429858424395
28.49514711126782
29.909689457344903
27.618428490628688
28.876735746968638
27.727380511393864
28.087561196091052
27.87738357617705
28.525380645375378
28.93061589304175
30.65148786386562
28.7732052863676
28.057381478490697
29.453585931334235
28.428732560941544
27.347491757276128
27.472579812825973
29.059961308425542
27.9465281784828
29.49787169418279
28.926257332622733
27.86367171489804
28.5578392004957
29.860523593002483
28.798448697276157
28.061151418487405
27.675984031718947
28.226364314621062
28.798448697276157
29.34723478614593
28.08931274518363
27.878986305365956
28.434072852786482
27.60220468061422
27.689065129091315
28.24639201658799
29.76612366872263
28.84566024853864
28.4448480021

28.82899757799678
28.50753858131402
27.554334590925173
28.639554081265533
29.051805799585626
28.15691815442266
29.19461676918139
28.182510769199062
28.46846364107821
28.349675372597524
28.271455793369835
28.170271918429457
27.918973824395703
28.179356335279365
29.389427400641477
27.898079782287397
27.67730044318818
27.6315888001311
27.967784740385255
29.757952920702756
29.910837991736976
28.639554081265533
27.894749778395997
29.725775995726444
28.784658618127676
28.896398657796322
28.954862370027893
27.78134202024244
27.24840483613777
28.100417124687958
28.044340012192865
27.451193359499012
27.98629045700771
28.842538916672055
27.869976176235603
30.478499310075616
28.914553225462342
29.58803278204483
29.95401329474393
27.875344871792915
28.132532027722288
29.906111958466354
27.57862040894469
27.76421313854663
27.960298696144754
30.12321010393015
27.63478310960362
29.445728602277484
28.095208606150848
27.643993538603002
28.132716870425398
28.33894264824456
27.24840483613777
27.967544664